In [1]:
% pip uninstall pyzmq


UsageError: Line magic function `%` not found.


In [2]:
%pip install matplotlib
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\kiraa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\kiraa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from matplotlib import cm

In [4]:
def my_logpdf(x, u, covar):
    """This function calculates the value of a multivariate Gaussian pdf at a given point x, with mean u and covariance covar

    Args:
        x : The point at which the pdf is evaluated. Nx1 column vector.
        u : The mean vector of the Gaussian distribution. Nx1 column vector.
        covar : The covariance matrix of the Gaussian distribution. NxN matrix.

    Returns:
        The value of the Gaussian pdf at x.
    """
    k = len(x)  # dimension
    a = np.transpose(x - u)
    b = np.linalg.inv(covar)
    c = x - u
    d = np.matmul(a, b)
    e = np.matmul(d, c)
    numer = np.exp(-0.5 * e)
    f = (2 * np.pi)**k
    g = np.linalg.det(covar)
    denom = np.sqrt(f * g)
    pdf = numer / denom
    return pdf

v = np.array([0.02, 0.02], dtype=np.float32)
covar = np.diag(v)

default = lambda x,y: 20*my_logpdf(np.array([x,y]),np.array([0,0]),covar)

goal = lambda X,Y: 30*((X-0)**2 + (Y-(-1))**2)

In [5]:
%matplotlib tk
n_points = 500
x = np.linspace(-1.5,1.5,n_points)
y = np.linspace(-1,1,n_points)

X, Y = np.meshgrid(x,y)

default_map = np.zeros((n_points,n_points))

for i in range(n_points):
    for j in range(n_points):
        default_map[j,i] = default(x[j],y[i])

In [9]:


dropwave = lambda X,Y,f,s,p,o,k,A: -np.abs(A*(o + np.cos(f*np.sqrt((X/k)**2+(Y/k)**2))/((s)*(X**2+Y**2)+p)))
dropwave_attenuation = lambda X,Y,f,s,p,o,k,A,gx,gy,zita: dropwave(X,Y,f,s,p,o,k,A) * np.exp(zita*((X-gx)**2+(Y-gy)**2))
f = 1
s = 0.5
p = 1
o = 1
k = 1
A = 1
zita = 1
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(2,1,1,projection='3d')

#p: Inversamente proprozionale al picco
#s: Inversamente proprozionale al cratere
#f: legato alla frequenza
#o: Un offset

f = 0.00001
p = 1
s = 0.5
o = 0
surf = ax.plot_surface(X,Y,dropwave_attenuation(X,Y,f,s,p,o,k,A,0,-1,zita) + default_map + goal(X,Y), cmap=cm.coolwarm)
# fix axis to robotarium size
ax.set_xlim(-1.5,1.5)
ax.set_ylim(-1,1)
ax.set_zlim(-10,100)

ax2 = fig.add_subplot(2,1,2)
gradient = np.gradient(dropwave_attenuation(X,Y,f,s,p,o,k,A,0,-1,zita) + default_map.T + goal(X,Y))
gradient_module = np.sqrt(gradient[0]**2 + gradient[1]**2)
mesh = ax2.pcolormesh(X,Y,gradient_module.T, cmap='spring', vmin=0, vmax=1)
fig.colorbar(mesh, ax=ax2)

In [10]:
import ipywidgets as widgets
from IPython.display import display

# Define the sliders
f_slider = widgets.FloatSlider(min=0.00001, max=250.0, step=0.00001, value=100, description='f:')
s_slider = widgets.FloatSlider(min=0.0001, max=1000.0, step=0.001, value=100, description='s:')
p_slider = widgets.FloatSlider(min=0.0001, max=1500.0, step=0.001, value=30.0, description='p:')
o_slider = widgets.FloatSlider(min=0.0, max=10.0, step=0.001, value=0.0, description='o:')
k_slider = widgets.FloatSlider(min=0.0, max=100.0, step=0.001, value=7.41, description='k:')
A_slider = widgets.FloatSlider(min=0.0, max=1000.0, step=0.001, value=37.16, description='A:')
zita_slider = widgets.FloatSlider(min=1.0, max=100.0, step=0.00001, value=1.0, description='zita:')
# Display the sliders
display(f_slider, s_slider, p_slider, o_slider, k_slider, A_slider, zita_slider)

# Update the function to use the slider values
def update_plot():
    f = f_slider.value
    s = s_slider.value
    p = p_slider.value
    o = o_slider.value
    k = k_slider.value
    A = A_slider.value
    zita = zita_slider.value
    ax.clear()
    surf = ax.plot_surface(X,Y,dropwave_attenuation(X,Y,f,s,p,o,k,A,0,-1,zita) + default_map.T + goal(X,Y), cmap=cm.coolwarm)
    ax.set_xlim(-1.5,1.5)
    ax.set_ylim(-1,1)
    ax.set_zlim(-5,100)
    ax2.clear()
    gradient = np.gradient(dropwave_attenuation(X,Y,f,s,p,o,k,A,0,-1,zita) + default_map.T + goal(X,Y))
    gradient_module = np.sqrt(gradient[0]**2 + gradient[1]**2)
    ax2.pcolormesh(X,Y,gradient_module.T, cmap='spring', vmin=0, vmax=1)


# Call the update function when a slider value changes
f_slider.observe(lambda change: update_plot(), names='value')
s_slider.observe(lambda change: update_plot(), names='value')
p_slider.observe(lambda change: update_plot(), names='value')
o_slider.observe(lambda change: update_plot(), names='value')
k_slider.observe(lambda change: update_plot(), names='value')
A_slider.observe(lambda change: update_plot(), names='value')
zita_slider.observe(lambda change: update_plot(), names='value')
# Initial plot
update_plot()

FloatSlider(value=100.0, description='f:', max=250.0, min=1e-05, step=1e-05)

FloatSlider(value=100.0, description='s:', max=1000.0, min=0.0001, step=0.001)

FloatSlider(value=30.0, description='p:', max=1500.0, min=0.0001, step=0.001)

FloatSlider(value=0.0, description='o:', max=10.0, step=0.001)

FloatSlider(value=7.41, description='k:', step=0.001)

FloatSlider(value=37.16, description='A:', max=1000.0, step=0.001)

FloatSlider(value=1.0, description='zita:', min=1.0, step=1e-05)